<a href="https://colab.research.google.com/github/100495802/G11.AA-495802-495702/blob/main/P1AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook contiene toda la parte de análisis y examinación de los datos, y se centra en la comprensión y resumen de éstos. En este notebook encontraremos todo el proceso de creacion del EDA, ajuste de hiperparámetros, selección del modelo, y resolución de las preguntas establecidas en el enunciado de la práctica.

# 1. Github y Carga de Datos

Primeramente, nos disponemos a realizar el [repositorio de Github](https://github.com/100495802/G11.AA-495802-495702.git) correspondiente al proyecto, el cual se puede consultar clickando en el enlace.

Seguidamente, cargamos los datos elegidos según el estándar especificado para la práctica (suma de los dos últimos dígitos del NIA de un alumno: _100495702 -> 0+2=2_).

## 1.1. Carga de datos

In [4]:
import pandas as pd
datos = pd.read_csv("attrition_availabledata_02.csv.gz", compression="gzip", sep=",")

# NOTEBOOK 1: EDA

En esta sección vamos a comentar la descripción de los distintos datos que hay en nuestra base de datos. Diferenciaremos que variable son categóricas, ordinales o numéricas; cuáles tienen alta cardinalidad, cuáles tienen valores faltantes; si hay columnas de ID; y si es un problema de regresión o de evaluación.

Mediante el atributo info() vamos a obtener los tipos de datos que hay en nuestra base de datos y asi poder evaluar los datos.

In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      2940 non-null   float64
 1   absences                 2940 non-null   int64  
 2   JobInvolvement           2940 non-null   int64  
 3   PerformanceRating        2940 non-null   int64  
 4   EnvironmentSatisfaction  2927 non-null   float64
 5   JobSatisfaction          2925 non-null   float64
 6   WorkLifeBalance          2911 non-null   float64
 7   Age                      2940 non-null   int64  
 8   BusinessTravel           2940 non-null   object 
 9   Department               2940 non-null   object 
 10  DistanceFromHome         2940 non-null   int64  
 11  Education                2940 non-null   int64  
 12  EducationField           2940 non-null   object 
 13  EmployeeCount            2940 non-null   int64  
 14  EmployeeID              

In [13]:
datos.describe()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
count,2940.000000,2940.000000,2940.000000,2940.000000,2927.000000,2925.000000,2911.000000,2940.000000,2940.000000,2940.000000,...,2940.000000,2926.000000,2940.000000,2940.0,2940.000000,2935.000000,2940.000000,2940.000000,2940.000000,2940.000000
mean,7.322768,12.706803,2.728571,3.154422,2.723266,2.746325,2.760907,36.861224,9.305102,2.906463,...,64379.826531,2.664388,15.187075,8.0,0.785034,11.363884,2.798639,7.095578,2.227891,4.191156
std,1.335600,5.533199,0.716167,0.361414,1.096170,1.104612,0.713539,9.286733,8.201638,1.023254,...,47021.197990,2.495153,3.661275,0.0,0.851343,7.897032,1.304166,6.161878,3.274101,3.627734
min,5.416880,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.000000,...,10090.000000,0.000000,11.000000,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.272786,8.000000,2.000000,3.000000,2.000000,2.000000,2.000000,30.000000,2.000000,2.000000,...,28750.000000,1.000000,12.000000,8.0,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000
50%,7.032627,13.000000,3.000000,3.000000,3.000000,3.000000,3.000000,35.000000,7.000000,3.000000,...,48340.000000,2.000000,14.000000,8.0,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000
75%,7.948416,17.000000,3.000000,3.000000,4.000000,4.000000,3.000000,43.000000,14.000000,4.000000,...,80080.000000,4.000000,18.000000,8.0,1.000000,16.000000,3.000000,10.000000,3.000000,7.000000
max,10.937261,24.000000,4.000000,4.000000,4.000000,4.000000,4.000000,60.000000,29.000000,5.000000,...,199990.000000,9.000000,25.000000,8.0,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000


Con los datos visualizados en el info, vamos a observar los valores de las variables más a fondo, haciendo hincapién en las distintas observaciones que se piden en la práctica, empezamos visualizando los valores de las variables númericas

In [32]:
datos.select_dtypes(include=['int64', 'float64']).value_counts()

,,,,,,,,,,,,,,,,,,,,,,,count
hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,EmployeeCount,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,
10.937261,2,4,3,4.0,3.0,3.0,33,8,2,1,2121,2,161840,0.0,13,8,1,10.0,2,9,0,8,1
10.695009,3,3,4,1.0,3.0,3.0,24,2,3,1,222,2,39190,1.0,21,8,0,6.0,3,6,1,2,1
10.693742,7,3,4,1.0,1.0,3.0,26,9,3,1,2967,1,162910,1.0,22,8,0,5.0,3,5,3,3,1
10.691539,3,3,3,1.0,1.0,3.0,38,2,4,1,1791,2,23680,6.0,11,8,1,17.0,3,15,4,12,1
10.681425,5,1,4,2.0,4.0,2.0,29,4,2,1,3508,2,65400,1.0,22,8,0,10.0,2,10,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5.466766,23,3,3,4.0,3.0,2.0,34,25,1,1,2344,1,22590,1.0,14,8,0,8.0,2,8,7,7,1
5.466699,22,1,3,1.0,3.0,3.0,58,27,3,1,3621,2,21870,1.0,18,8,1,2.0,1,2,2,2,1
5.461962,23,2,3,2.0,3.0,2.0,33,9,2,1,615,1,20970,1.0,14,8,2,10.0,5,10,0,1,1


Ahora hacemos lo mismo pero con las variables cadenas de texto

In [19]:
datos.select_dtypes(include=["object"]).value_counts()

BusinessTravel  Department              EducationField  Gender  JobRole                MaritalStatus  Over18  Attrition
Travel_Rarely   Research & Development  Life Sciences   Male    Laboratory Technician  Married        Y       No           40
                                                                Sales Executive        Married        Y       No           37
                                                        Female  Sales Executive        Married        Y       No           34
                                                        Male    Research Scientist     Single         Y       No           30
                                                                                       Married        Y       No           29
                                                                                                                           ..
                Sales                   Medical         Female  Sales Executive        Single         Y       Yes           1
Non-Travel      Research & Development  Life Sciences   Female  Sales Representative   Married        Y       Yes           1
                                                        Male    Human Resources        Single         Y       Yes           1
                                                                Laboratory Technician  Divorced       Y       No            1
                                                                Research Scientist     Married        Y       No            1
Name: count, Length: 726, dtype: int64

In [31]:
for columna in datos.columns:
    print(columna)
    print(datos[columna].unique())

hrs
[10.06004797  9.43767068  7.9009315  ...  9.1069199   6.53264502
  6.54273873]
absences
[ 6  2 20 19  8  4 10 18  5 14 16 11 12 15 17  3 13  9 22 21  7 23  1 24]
JobInvolvement
[3 2 4 1]
PerformanceRating
[4 3]
EnvironmentSatisfaction
[ 2.  3.  4.  1. nan]
JobSatisfaction
[ 4.  2.  3.  1. nan]
WorkLifeBalance
[ 1.  3.  2.  4. nan]
Age
[31 33 35 28 26 40 44 36 30 49 24 32 41 23 42 25 34 39 38 37 21 58 45 50
 29 57 43 48 55 46 19 27 20 47 51 53 56 59 54 52 18 60 22]
BusinessTravel
['Travel_Frequently' 'Non-Travel' 'Travel_Rarely']
Department
['Research & Development' 'Sales' 'Human Resources']
DistanceFromHome
[ 7 18 15 10  8  9 25  2  4  1  6 29  5 20 23 13 22 11 28  3 21 16 17 12
 19 24 27 26 14]
Education
[4 2 3 1 5]
EducationField
['Life Sciences' 'Medical' 'Other' 'Technical Degree' 'Marketing'
 'Human Resources']
EmployeeCount
[1]
EmployeeID
[ 779 3881 1024 ... 3304 2826 1486]
Gender
['Male' 'Female']
JobLevel
[1 2 4 3 5]
JobRole
['Laboratory Technician' 'Healthcare Representat

Analizando el texto impreso podemos observar que variables presentan elevada cardinalidad. Vamos a considerar de alta cardinalidad las variables que tienen tantos valores que no se pueden imprimir todos, es decir, las que presentan los tres puntos (...) en la impresión. Estas variables son: Monthly Income, EmployeID y hours.

A continuación podemos comprobar si existen valores nulos en las variables y cuantos hay en cada una de ellas.

In [17]:
nulos = datos.isnull().sum()
nulos[nulos > 0]

,0
EnvironmentSatisfaction,13
JobSatisfaction,15
WorkLifeBalance,29
NumCompaniesWorked,14
TotalWorkingYears,5


Como observamos tenemos 5 variables con valores nulos en sus filas. Teniendo en cuenta que en total hay 2900 filas en la tabla se pueden hacer fácilmente los cálculos de cuántas filas vacias hay por variable.

Una vez analizado todo lo anterior podemos buscar que posibles columnas actúan como id en la base de datos. Observando los nombres de las variables una posible opción sería EmployeeID aunque queremos comprobar si actúa como ID alguna otra.

In [33]:
for columna in datos.columns:
    if datos[columna].nunique() == len(datos):
        print(f"La columna '{columna}' es un posible identificador único.")

La columna 'EmployeeID' es un posible identificador único.


Por último, el problema que se nos propone es de clasificación, pues contamos con la variable Attrition que determina si un empleado serñia propenso a abandonar la empresa en función del resto de datos asociados. Vamos a comprobar si se trata de un problema desbalanceado o no.

In [39]:
datos["Attrition"].value_counts()

,count
Attrition,
No,2466
Yes,474


Como observamos, el conteo de datos nos demuestra que es un problema desbalanceado hacia el no, siendo más del 80% de las clasificaciones de un tipo